In [1]:
import os
import cv2
import shutil
import numpy as np
import rasterio

# List all files
img_dir = "data/images"
mask_dir = "data/masks"

img_files = os.listdir(img_dir)
mask_files = os.listdir(mask_dir)

os.makedirs(img_dir + "/augmented", exist_ok=True)

# Process each file
for file in img_files:
    # Check if the file is a TIFF image
    if file.endswith(".tif"):
        # Load the image
        image_path = os.path.join(img_dir, file)
        image = cv2.imread(image_path)

        # Adjust brightness and contrast
        alpha = 1.2  # Controls brightness
        beta = 20    # Controls contrast
        adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

        # Convert to grayscale
        gray_image = cv2.cvtColor(adjusted_image, cv2.COLOR_BGR2GRAY)

        # Increase sharpness
        sharpened_image = cv2.filter2D(gray_image, -1, np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]))

        # Create a 3-band RGB image from the grayscale image
        three_band_gray = cv2.cvtColor(sharpened_image, cv2.COLOR_GRAY2BGR)

        # Save the augmented image with "-aug" extension
        augmented_image_path = os.path.join(img_dir, 'augmented', file[:-4] + "-aug.tif")
        cv2.imwrite(augmented_image_path, adjusted_image)

        # Save the gray augmented image with "-aug-gray" extension
        gray_augmented_image_path = os.path.join(img_dir, 'augmented', file[:-4] + "-aug-gray.tif")
        cv2.imwrite(gray_augmented_image_path, three_band_gray)

        # Duplicate the mask file for the augmented images
        mask_path = os.path.join(mask_dir, file)
        augmented_mask_path = os.path.join(mask_dir, file[:-4] + "-aug.tif")
        shutil.copy(mask_path, augmented_mask_path)

        # Duplicate the mask file for the gray augmented images
        mask_path = os.path.join(mask_dir, file)
        augmented_mask_path = os.path.join(mask_dir, file[:-4] + "-aug-gray.tif")
        shutil.copy(mask_path, augmented_mask_path)

        # Copy the metadata from the original image to the augmented image
        with rasterio.open(image_path) as src:
            metadata = src.profile
            metadata.update(count=3, dtype='uint8')

            with rasterio.open(augmented_image_path, 'w', **metadata) as dst:
                dst.write(adjusted_image.transpose(2, 0, 1))

        # Copy the metadata from the original image to the gray image
        with rasterio.open(image_path) as src:
            metadata = src.profile
            metadata.update(count=3, dtype='uint8')

            with rasterio.open(gray_augmented_image_path, 'w', **metadata) as dst:
                dst.write(three_band_gray.transpose(2, 0, 1))
